# Visualizing the progress of a language migration

or: how to show non-developers that we're working under the hood of a software system successfully

### Take a look at the raw dataset

In [ ]:
!head ../dataset/angular_file_commit_history.csv

### Read in the data from Git log

In [ ]:
import pandas as pd

log = pd.read_csv("../dataset/angular_file_commit_history.csv", parse_dates=[1])
log.head()

### Get the info about the programming language used

In [ ]:
log['lang'] = log['file_path'].str.rsplit(".", n=1).str[-1]
log.head()

### Map added and deleted lines to numbers

In [ ]:
log.loc[log['action'] == "A", "count"] = 1
log.loc[log['action'] == "D", "count"] = -1
log.head()

### Group changes to weekly numbers

In [ ]:
log_timed = log.groupby([pd.Grouper(key="timestamp", freq="1w"), "lang"])['count'].sum()
log_timed.head()

### Calculate data for showing the progress

In [ ]:
log_timed_unstacked = log_timed.unstack().fillna(0).cumsum()
log_timed_unstacked.head()

### Visualize the progress of the migration

In [ ]:
log_timed_unstacked.plot();

### Get into more details regarding the first migration

In [ ]:
log_timed_unstacked.loc['2015':'2016'].plot(title="Migration progress from JavaScript to TypeScript", ylabel="files");

### End